# Orchestrating Jobs, Model Registration, and Continuous Deployment with Amazon SageMaker in a secure environment

Amazon SageMaker offers Machine Learning application developers and Machine Learning operations engineers the ability to orchestrate SageMaker jobs and author reproducible Machine Learning pipelines, deploy custom-build models for inference in real-time with low latency or offline inferences with Batch Transform, and track lineage of artifacts. You can institute sound operational practices in deploying and monitoring production workflows, deployment of model artifacts, and track artifact lineage through a simple interface, adhering to safety and best-practice paradigmsfor Machine Learning application development.

The SageMaker Pipelines service supports a SageMaker Machine Learning Pipeline Domain Specific Language (DSL), which is a declarative Json specification. This DSL defines a Directed Acyclic Graph (DAG) of pipeline parameters and SageMaker job steps. The SageMaker Python Software Developer Kit (SDK) streamlines the generation of the pipeline DSL using constructs that are already familiar to engineers and scientists alike.

The SageMaker Model Registry is where trained models are stored, versioned, and managed. Data Scientists and Machine Learning Engineers can compare model versions, approve models for deployment, and deploy models from different AWS accounts, all from a single Model Registry. SageMaker enables customers to follow the best practices with ML Ops and getting started right. Customers are able to standup a full ML Ops end-to-end system with a single API call.

## SageMaker Pipelines

Amazon SageMaker Pipelines support the following activites:

* Pipelines - A Directed Acyclic Graph of steps and conditions to orchestrate SageMaker jobs and resource creation.
* Processing Job steps - A simplified, managed experience on SageMaker to run data processing workloads, such as feature engineering, data validation, model evaluation, and model interpretation.
* Training Job steps - An iterative process that teaches a model to make predictions by presenting examples from a training dataset.
* Conditional step execution - Provides conditional execution of branches in a pipeline.
* Registering Models - Creates a model package resource in the Model Registry that can be used to create deployable models in Amazon SageMaker.
* Creating Model steps - Create a model for use in transform steps or later publication as an endpoint.
* Parametrized Pipeline executions - Allows pipeline executions to vary by supplied parameters.
* Transform Job steps - A batch transform to preprocess datasets to remove noise or bias that interferes with training or inference from your dataset, get inferences from large datasets, and run inference when you don't need a persistent endpoint.

## Running SageMaker workloads in a secure network configuration

This data science environment is deployed in a secure network configuration with private VPC, private subnets and security groups. 

To ensure and enforce the usage of this secure environment, we implement preventive security controls using IAM policy for SageMaker execution role.

This following IAM policy code enforce usage of VPC isolation (with private subnets) for SageMaker notebook instances, processing, training, and tuning jobs, as well as for models.

```json
{
    "Action": [
        "sagemaker:CreateNotebookInstance",
        "sagemaker:CreateHyperParameterTuningJob",
        "sagemaker:CreateProcessingJob",
        "sagemaker:CreateTrainingJob",
        "sagemaker:CreateModel"
    ],
    "Resource": "*",
    "Effect": "Deny",
    "Condition": {
        "Null": {
            "sagemaker:VpcSubnets": "true"
        }
    }
}
```

Any SageMaker workload which is running under SageMaker execution role is enforced to use VPC configuration for these tasks. If you try to create any job without specifying a network configuration, an `AccessDeny` exception will be raised.
The following sections show step-by-step how to configure the secure execution of SageMaker workloads.



## Implementing preventive security controls using IAM condition keys

[SageMaker-specific IAM condition keys](https://docs.aws.amazon.com/IAM/latest/UserGuide/list_amazonsagemaker.html) can be used to improve security by preventing resources from being created without security controls. For example:

```json
"Condition": {
   "StringEquals": {
      "sagemaker:RootAccess": "Disabled"
   }
}
```

Security-specific examples of the condition keys:
- `sagemaker:DirectInternetAccess`
- `sagemaker:InterContainerTrafficEncryption`
- `sagemaker:NetworkIsolation`
- `sagemaker:RootAccess`
- `sagemaker:VPCSecurityGroupIds`: should be set to a pre-created security group configured with the necessary controls
- `sagemaker:VPCSubnets`
- `sagemaker:VolumeKmsKey`
- `sagemaker:OutputKmsKey`

### Example: Enforce usage of network isolation mode
To enforce usage of resource secure configuration, you can add the following policy to the SageMaker excecution role:
```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Deny",
            "Action": [
                "sagemaker:Create*"
            ],
            "Resource": "*",
            "Condition": {
                "StringNotEqualsIfExists": {
                    "sagemaker:NetworkIsolation": "true"
                }
            }
        }
    ]
}
```

The policy denies creation of any component (processing or traning job, endpoint, transform job) if the `sagemaker:NetworkIsolation` parameter is not set to `true`. This applies only to the components which have this parameter.
Similarly you can add validation of any other SageMaker service-specific condition keys.

## Layout of the SageMaker ModelBuild Project Template

The template provides a starting point for bringing your SageMaker Pipeline development to production.

```
|-- buildspec.yml
|-- CONTRIBUTING.md
|-- dataset
|   `-- abalone-dataset.csv
|-- pipelines
|   |-- abalone
|   |   |-- evaluate.py
|   |   |-- __init__.py
|   |   |-- pipeline.py
|   |   `-- preprocess.py
|   |-- get_pipeline_definition.py
|   |-- __init__.py
|   |-- run_pipeline.py
|   |-- _utils.py
|   `-- __version__.py
|-- README.md
|-- sagemaker-pipeline.ipynb
|-- sagemaker-pipelines-project.ipynb
|-- setup.cfg
|-- setup.py
|-- tests
|   `-- test_pipelines.py
`-- tox.ini
```

A description of some of the artifacts is provided below:
<br/><br/>
Your codebuild execution instructions:
```
|-- buildspec.yml
```
<br/><br/>
Your pipeline artifacts, which includes a pipeline module defining the required `get_pipeline` method that returns an instance of a SageMaker pipeline, a preprocessing script that is used in feature engineering, and a model evaluation script to measure the Mean Squared Error of the model that's trained by the pipeline:

```
|-- pipelines
|   |-- abalone
|   |   |-- evaluate.py
|   |   |-- __init__.py
|   |   |-- pipeline.py
|   |   `-- preprocess.py

```
<br/><br/>
Utility modules for getting pipeline definition jsons and running pipelines:

```
|-- pipelines
|   |-- get_pipeline_definition.py
|   |-- __init__.py
|   |-- run_pipeline.py
|   |-- _utils.py
|   `-- __version__.py
```
<br/><br/>
Python package artifacts:
```
|-- setup.cfg
|-- setup.py
```
<br/><br/>
A stubbed testing module for testing your pipeline as you develop:
```
|-- tests
|   `-- test_pipelines.py
```
<br/><br/>
The `tox` testing framework configuration:
```
`-- tox.ini
```

### A SageMaker Pipeline

The pipeline that we create follows a typical Machine Learning Application pattern of pre-processing, training, evaluation, and conditional model registration and publication, if the quality of the model is sufficient.

![A typical ML Application pipeline](img/pipeline-full.png)

### Get sagemaker package

In [ ]:
!pip install --disable-pip-version-check -q sagemaker==2.47.1

In [ ]:
!python --version

In [ ]:
import boto3
import sagemaker
import sagemaker.session
import json

print(f"SageMaker version: {sagemaker.__version__}")

### Getting some constants

We get some constants from the local execution environment.


In [ ]:
sm = boto3.client("sagemaker")
ssm = boto3.client("ssm")

def get_environment(project_name, ssm_params):
    r = sm.describe_domain(
            DomainId=sm.describe_project(
                ProjectName=project_name
                )["CreatedBy"]["DomainId"]
        )
    del r["ResponseMetadata"]
    del r["CreationTime"]
    del r["LastModifiedTime"]
    r = {**r, **r["DefaultUserSettings"]}
    del r["DefaultUserSettings"]

    i = {
        **r,
        **{t["Key"]:t["Value"] 
            for t in sm.list_tags(ResourceArn=r["DomainArn"])["Tags"] 
            if t["Key"] in ["EnvironmentName", "EnvironmentType"]}
    }

    for p in ssm_params:
        try:
            i[p["VariableName"]] = ssm.get_parameter(Name=f"{i['EnvironmentName']}-{i['EnvironmentType']}-{p['ParameterName']}")["Parameter"]["Value"]
        except:
            i[p["VariableName"]] = ""

    return i

<div class="alert alert-info"> 💡 <strong> Get environment variables </strong>

Set the <b>`project_name`</b> to the name of the current SageMaker project.
Various environment data is loaded and shown. We need these data later for a proper network parameters setup.
</div>

In [ ]:
# use the current project name
project_name = <PROJECT NAME>
project_id = sm.describe_project(ProjectName=project_name)['ProjectId']

# Dynamically load environmental SSM parameters - provide the list of the variables to load from SSM parameter store
ssm_parameters = [
    {"VariableName":"DataBucketName", "ParameterName":"data-bucket-name"},
    {"VariableName":"ModelBucketName", "ParameterName":"model-bucket-name"},
    {"VariableName":"S3KmsKeyId", "ParameterName":"kms-s3-key-arn"},
    {"VariableName":"EbsKmsKeyArn", "ParameterName":"kms-ebs-key-arn"},
    {"VariableName":"PipelineExecutionRole", "ParameterName":"sm-pipeline-execution-role-arn"},
]

env_data = get_environment(project_name=project_name, ssm_params=ssm_parameters)
print(f"Environment data:\n{json.dumps(env_data, indent=2)}")

Set some constants:

In [ ]:
from pipelines.abalone.pipeline import get_session

sagemaker_session = get_session(boto3.Session().region_name, env_data["DataBucketName"])

region = boto3.Session().region_name

print(f"Region: {region}")

# Change these to reflect your project/business name or if you want to separate ModelPackageGroup/Pipeline from the rest of your team
model_package_group_name = f"AbaloneModelPackageGroup-Example"
pipeline_name = f"AbalonePipeline-Example"

### Get the pipeline instance

Here we get the pipeline instance from your pipeline module (`pipelines/abalone/pipeline.py`) so that we can work with it.

In [ ]:
from pipelines.abalone.pipeline import get_pipeline

# get_pipeline function retrieves the values of environment parameters from Systems Manager parameter store and SageMaker domain metadata
pipeline = get_pipeline(
    region=region,
    project_name=project_name,
    model_package_group_name=model_package_group_name,
    pipeline_name=pipeline_name,
    base_job_prefix="Abalone"
)

### Submit the pipeline to SageMaker and start execution

Let's submit our pipeline definition to the workflow service. The pipeline role passed in is set by `env_data["PipelineExecutionRole"]` and will be used by the workflow service to create all the jobs defined in the steps.

In [ ]:
print(f"Use this role to run the pipeline: {env_data['PipelineExecutionRole']}")
pipeline.upsert(role_arn=env_data["PipelineExecutionRole"])

We'll start the pipeline, accepting all the default parameters.

Values can also be passed into these pipeline parameters on starting of the pipeline, and will be covered later. 

Now print out the pipeline definition. Please note the network configuration parameters for pre-processing, training and evaluation steps:

Pre-processing and evaluation steps:
```json
 "NetworkConfig": {
          "EnableInterContainerTrafficEncryption": true,
          "EnableNetworkIsolation": false,
          "VpcConfig": {
            "SecurityGroupIds": [
              "sg-09d0ae98a2abd26f5"
            ],
            "Subnets": [
              "subnet-0a338babaa635e3e6"
            ]
          }
        }
```

Training step:
```json
"EnableInterContainerTrafficEncryption": true,
"VpcConfig": {
          "SecurityGroupIds": [
            "sg-09d0ae98a2abd26f5"
          ],
          "Subnets": [
            "subnet-0a338babaa635e3e6"
          ]
        }
```

In [ ]:
parsed = json.loads(pipeline.definition())
print(json.dumps(parsed, indent=2, sort_keys=True))

The following line starts the pipeline execution. In this specific example it runs for about 13 minutes. You can follow the execution progresss in Studio pipeline widget:
  
![studio pipelines](img/studio-pipeline-execution.png)

In [ ]:
execution = pipeline.start()

### Pipeline Operations: examining and waiting for pipeline execution

Now we describe execution instance and list the steps in the execution to find out more about the execution.

In [ ]:
execution.describe()

We can wait for the execution by invoking `wait()` on the execution:

In [ ]:
execution.wait()

We can list the execution steps to check out the status and artifacts:

In [ ]:
execution.list_steps()

### Parametrized Executions

We can run additional executions of the pipeline specifying different pipeline parameters. The parameters argument is a dictionary whose names are the parameter names, and whose values are the primitive values to use as overrides of the defaults.

Of particular note, based on the performance of the model, we may want to kick off another pipeline execution, but this time on a compute-optimized instance type and set the model approval status automatically be "Approved". This means that the model package version generated by the `RegisterModel` step will automatically be ready for deployment through CI/CD pipelines, such as with SageMaker Projects.

In [ ]:
execution = pipeline.start(
    parameters=dict(
        ProcessingInstanceType="ml.c5.xlarge",
        ModelApprovalStatus="Approved",
    )
)

In [ ]:
execution.wait()

In [ ]:
execution.list_steps()

## Clean up

### Delete SageMaker project
This will delete the associated CloudFormation stack and CodeCommit repository

In [ ]:
print(f"Deleting project {project_name}:{sm.delete_project(ProjectName=project_name)}")

### Delete project S3 bucket 
This will remove all files and S3 bucket

In [ ]:
!aws s3 rb s3://sm-mlops-cp-{project_name}-{project_id} --force

## Release resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>